In [4]:
import pandas as pd
import numpy as np
import re
import os
from collections import defaultdict
from scipy.spatial.distance import jensenshannon

In [5]:
def extract_cyclic_attractors(biological_attractor_file):
    attrs = biological_attractor_file.iloc[:, 1:]
    grouped_attrs = defaultdict(list)
    pattern = re.compile(r"^(.*)_s\d+$")
    for col in attrs.columns:
        match = pattern.match(col)
        if match:
            group_name = match.group(1)
            grouped_attrs[group_name].append(col)
            
    biological_cycles = []
    for group_name, cols in grouped_attrs.items():
        cycle = []
        for col in cols:
            binary_str = ''.join(str(x) for x in attrs[col].astype(int).tolist())
            reversed_str = binary_str[::-1]
            decimal_val = int(reversed_str, 2)
            cycle.append(str(decimal_val))
        biological_cycles.append(cycle)
    return biological_cycles


def extract_fxd_pt_attractor(biological_attractor_file):
    attrs = biological_attractor_file.iloc[:, 1:]
    biological_fxd_pts = []
    for col in attrs.columns:
        binary_str = ''.join(str(x) for x in attrs[col].astype(int).tolist())
        reversed_str = binary_str[::-1]
        decimal_val = int(reversed_str, 2)
        biological_fxd_pts.append(str(decimal_val))
    return biological_fxd_pts



def bio_and_thresh_maj_rule_attractors(model_no):
    pub_and_TMR_att_dict = {category:{'fixed_points': None, 'cycles': None} for category in ['published', 'IMR', 'BMR']}
    
    pub_rule_attr_df = pd.read_csv(f'../input/att_details_all_files/published_rule_attractors_model_{model_no}.tsv', sep = '\t')
    IMR_attr_df = pd.read_csv(f'../input/att_details_all_files/IMR_rule_attractors_model_{model_no}.tsv', sep = '\t')
    BMR_attr_df = pd.read_csv(f'../input/att_details_all_files/BMR_rule_attractors_model_{model_no}.tsv', sep = '\t')
    
    pub_and_TMR_att_dict['published']['fixed_points'] = eval(pub_rule_attr_df['fixed_points'][0])
    pub_and_TMR_att_dict['published']['cycles'] = eval(pub_rule_attr_df['cycles'][0])

    pub_and_TMR_att_dict['IMR']['fixed_points'] = eval(IMR_attr_df['fixed_points'][0])
    pub_and_TMR_att_dict['IMR']['cycles'] = eval(IMR_attr_df['cycles'][0])
    
    pub_and_TMR_att_dict['BMR']['fixed_points'] = eval(BMR_attr_df['fixed_points'][0])
    pub_and_TMR_att_dict['BMR']['cycles'] = eval(BMR_attr_df['cycles'][0])
   
    return pub_and_TMR_att_dict

def cyclic_att_intersection(cycle_list_1, cycle_list_2):
    matched_elements = []
    indices_in_cycle_list_1 = []
    indices_in_cycle_list_2 = []
    
    for i, a in enumerate(cycle_list_1):
        for j, b in enumerate(cycle_list_2):
            if len(a) == len(b) and any(a == b[k:] + b[:k] for k in range(len(b))):
                matched_elements.append(a)
                indices_in_cycle_list_1.append(i)
                indices_in_cycle_list_2.append(j)
    
    return matched_elements, indices_in_cycle_list_1, indices_in_cycle_list_2



def fxd_att_intersection_info(fxd_pt_list_1, fxd_pt_list_2):
    matched_elements = []
    indices_in_fxd_pt_list_1 = []
    indices_in_fxd_pt_list_2 = []

    for i, a in enumerate(fxd_pt_list_1):
        if a in fxd_pt_list_2:
            j = fxd_pt_list_2.index(a)
            matched_elements.append(a)
            indices_in_fxd_pt_list_1.append(i)
            indices_in_fxd_pt_list_2.append(j)

    return matched_elements, indices_in_fxd_pt_list_1, indices_in_fxd_pt_list_2

# Attractor recovery score

In [6]:
model_nums = [7,8,10,22,23,31,40,61,63,67,69,74,85,86,88,95,99,100,109,110,133,200,208,212]
jaccard_data_dict = {'model_no':[], 'tot_nodes':[], 'max_indeg':[], 'mean_indeg':[], 'original_mod_jaccard':[], 'IMR_maj_jaccard': [], 'BMR_maj_jaccard':[]}
for model in model_nums:
    all_att = bio_and_thresh_maj_rule_attractors(model)
    jaccard_data_dict['model_no'].append(model)
    
    func_types = pd.read_csv(f'../../network_generation/output/published_network_info/model_{model}/func_type_details_model_{model}.tsv', sep = '\t')
    jaccard_data_dict['tot_nodes'].append(len(func_types))
    jaccard_data_dict['max_indeg'].append(int(func_types['No. of inputs'].max()))
    jaccard_data_dict['mean_indeg'].append(round(func_types['No. of inputs'].mean(), 3))
    
    if model in [23,95]:
        fxd_pt_bio_meaning_attract_df = pd.read_csv(f'../input/biological_attractor_files/attractor_model_{model}.tsv', sep = '\t')
        cycle_bio_meaning_attract_df = pd.read_csv(f'../input/biological_attractor_files/cyclic_attractor_model_{model}.tsv', sep = '\t')
        p_fxd = extract_fxd_pt_attractor(fxd_pt_bio_meaning_attract_df)
        p_cycle = extract_cyclic_attractors(cycle_bio_meaning_attract_df)
        p_m_jaccard = (len(p_fxd)+len(p_cycle))/(len(all_att['published']['fixed_points'])+len(all_att['published']['cycles']))
        
        p_n_IMR_intersection = len(fxd_att_intersection_info(p_fxd, all_att['IMR']['fixed_points'])[0]) + len(cyclic_att_intersection(p_cycle, all_att['IMR']['cycles'])[0])
        p_n_IMR_union = len(all_att['IMR']['fixed_points'])+len(all_att['IMR']['cycles'])+len(p_fxd)+len(p_cycle)-p_n_IMR_intersection
        p_n_IMR_jaccard = p_n_IMR_intersection/p_n_IMR_union
        
        p_n_BMR_intersection = len(fxd_att_intersection_info(p_fxd, all_att['BMR']['fixed_points'])[0]) + len(cyclic_att_intersection(p_cycle, all_att['BMR']['cycles'])[0])
        p_n_BMR_union = len(all_att['BMR']['fixed_points'])+len(all_att['BMR']['cycles'])+len(p_fxd)+len(p_cycle)-p_n_BMR_intersection
        p_n_BMR_jaccard = p_n_BMR_intersection/p_n_BMR_union
        
        
    elif model in [31, 69, 109]:
        cycle_bio_meaning_attract_df = pd.read_csv(f'../input/biological_attractor_files/cyclic_attractor_model_{model}.tsv', sep = '\t')
        p_cycle = extract_cyclic_attractors(cycle_bio_meaning_attract_df)
        
        p_m_jaccard = len(p_cycle)/(len(all_att['published']['fixed_points'])+len(all_att['published']['cycles']))
        
        p_n_IMR_intersection = len(cyclic_att_intersection(p_cycle, all_att['IMR']['cycles'])[0])
        p_n_IMR_union = len(all_att['IMR']['fixed_points'])+len(all_att['IMR']['cycles'])+ len(p_cycle)-p_n_IMR_intersection
        p_n_IMR_jaccard = p_n_IMR_intersection/p_n_IMR_union
        
        p_n_BMR_intersection = len(cyclic_att_intersection(p_cycle, all_att['BMR']['cycles'])[0])
        p_n_BMR_union = len(all_att['BMR']['fixed_points'])+len(all_att['BMR']['cycles'])+ len(p_cycle)-p_n_BMR_intersection
        p_n_BMR_jaccard = p_n_BMR_intersection/p_n_BMR_union

            
    else:
        fxd_pt_bio_meaning_attract_df = pd.read_csv(f'../input/biological_attractor_files/attractor_model_{model}.tsv', sep = '\t')
        p_fxd = extract_fxd_pt_attractor(fxd_pt_bio_meaning_attract_df)
        
        p_m_jaccard = len(p_fxd)/(len(all_att['published']['fixed_points'])+len(all_att['published']['cycles']))
        
        p_n_IMR_intersection = len(fxd_att_intersection_info(p_fxd, all_att['IMR']['fixed_points'])[0])
        p_n_IMR_union = len(all_att['IMR']['fixed_points'])+len(all_att['IMR']['cycles'])+len(p_fxd)-p_n_IMR_intersection
        p_n_IMR_jaccard = p_n_IMR_intersection/p_n_IMR_union
        
        p_n_BMR_intersection = len(fxd_att_intersection_info(p_fxd, all_att['BMR']['fixed_points'])[0])
        p_n_BMR_union = len(all_att['BMR']['fixed_points'])+len(all_att['BMR']['cycles'])+len(p_fxd)-p_n_BMR_intersection
        p_n_BMR_jaccard = p_n_BMR_intersection/p_n_BMR_union
        
        
    jaccard_data_dict['original_mod_jaccard'].append(p_m_jaccard)
    jaccard_data_dict['IMR_maj_jaccard'].append(p_n_IMR_jaccard)
    jaccard_data_dict['BMR_maj_jaccard'].append(p_n_BMR_jaccard)
    
df = pd.DataFrame(jaccard_data_dict)
df['att_recovery_score_IMR'] = df['IMR_maj_jaccard']/df['original_mod_jaccard']
df['att_recovery_score_BMR'] = df['BMR_maj_jaccard']/df['original_mod_jaccard']
df.to_csv('../output/attractor_recovery.tsv', sep = '\t', index = False)

In [7]:
df

,model_no,tot_nodes,max_indeg,mean_indeg,original_mod_jaccard,IMR_maj_jaccard,BMR_maj_jaccard,att_recovery_score_IMR,att_recovery_score_BMR
0,7,5,4,2.800,1.000000,1.000000,0.400000,1.000000,0.400000
1,8,28,4,1.714,0.278351,0.029484,0.000025,0.105924,0.000089
2,10,15,7,2.600,0.333333,0.000000,0.015504,0.000000,0.046512
3,22,22,7,2.000,0.065574,0.007874,0.000120,0.120079,0.001831
4,23,10,6,3.500,1.000000,0.000000,0.014085,0.000000,0.014085
5,31,9,4,2.111,0.500000,0.000000,0.000000,0.000000,0.000000
6,40,23,5,1.652,0.090909,0.003284,0.000289,0.036125,0.003179
7,61,26,9,3.115,0.333333,0.000000,0.000000,0.000000,0.000000
8,63,13,4,1.923,0.888889,0.225806,0.042735,0.254032,0.048077
9,67,13,3,1.615,0.619048,0.122642,0.033898,0.198113,0.054759


# Basin recovery using JS distance

In [8]:
att_details_publ = pd.read_csv('../input/all_published_rule_attprops.tsv', sep = '\t')
att_details_IMR = pd.read_csv('../input/all_IMR_rule_attprops.tsv', sep = '\t')
att_details_BMR = pd.read_csv('../input/all_BMR_rule_attprops.tsv', sep = '\t')

In [9]:
def fxd_pt_basin_sizes_for_JS_test(model, att_details_publ, att_details_IMR, att_details_BMR):
    gold_std_basin_sizes_fxd_pts, spurious_bio_basins_fxd_pts,  IMR_model_basin_sizes_fxd_pts, BMR_model_basin_sizes_fxd_pts = [], [], [], []
        
    relev_bio = att_details_publ[att_details_publ['model_number'] == model]
    relev_IMR = att_details_IMR[att_details_IMR['model_number'] == model]
    relev_BMR = att_details_BMR[att_details_BMR['model_number'] == model]
      
    biological_fxd_pt_path = f"../input/biological_attractor_files/attractor_model_{model}.tsv"
    if os.path.exists(biological_fxd_pt_path):
        fxd_pt_bio_meaning_attract_df = pd.read_csv(biological_fxd_pt_path, sep='\t')
        p_fxd = extract_fxd_pt_attractor(fxd_pt_bio_meaning_attract_df)
    else:
        p_fxd = []
    
    all_att = bio_and_thresh_maj_rule_attractors(model)
    
    # Computing the gold standard for the published model
    p_m_intersection = fxd_att_intersection_info(p_fxd, all_att['published']['fixed_points'])
    gold_standard_indices_in_bio = p_m_intersection[2]
    spurious_bio_att_indices_in_bio = [ele for ele in np.arange(len(all_att['published']['fixed_points'])) if ele not in gold_standard_indices_in_bio]
    
    for val in gold_standard_indices_in_bio:
        matched_values = relev_bio.loc[relev_bio['attractor_number'] == val+1, 'attractor_basin_size'].tolist()
        gold_std_basin_sizes_fxd_pts.extend(matched_values)
        
    for val in spurious_bio_att_indices_in_bio:
        matched_values = relev_bio.loc[relev_bio['attractor_number'] == val+1, 'attractor_basin_size'].tolist()
        spurious_bio_basins_fxd_pts.extend(matched_values)
    
    gold_std_basin_sizes_fxd_pts_for_bio = gold_std_basin_sizes_fxd_pts + [0]*len(spurious_bio_basins_fxd_pts)
    bio_model_basin_sizes_fxd_pts = gold_std_basin_sizes_fxd_pts + spurious_bio_basins_fxd_pts
    
    
    # Computing the gold standard for the ising model
    p_n_IMR_intersection = fxd_att_intersection_info(p_fxd, all_att['IMR']['fixed_points'])
    gold_std_basin_sizes_fxd_pts_for_IMR = ([gold_std_basin_sizes_fxd_pts[i] for i in p_n_IMR_intersection[1]]
                                            + [gold_std_basin_sizes_fxd_pts[i] for i in range(len(gold_std_basin_sizes_fxd_pts)) if i not in p_n_IMR_intersection[1]]
                                            + [0]*(len(all_att['IMR']['fixed_points']) - len(p_n_IMR_intersection[1]))
    )
   
    for val in p_n_IMR_intersection[2]:
        matched_values = relev_IMR.loc[relev_IMR['attractor_number'] == val+1, 'attractor_basin_size'].tolist()
        IMR_model_basin_sizes_fxd_pts.extend(matched_values)
    
        
    IMR_model_basin_sizes_fxd_pts = IMR_model_basin_sizes_fxd_pts + [0]*(len(gold_std_basin_sizes_fxd_pts) - len(p_n_IMR_intersection[2]))
    
    IMR_indices_of_the_extra_attr = [ele for ele in np.arange(len(all_att['IMR']['fixed_points'])) if ele not in p_n_IMR_intersection[2]]
    
    for val in IMR_indices_of_the_extra_attr:
        matched_values = relev_IMR.loc[relev_IMR['attractor_number'] == val+1, 'attractor_basin_size'].tolist()
        IMR_model_basin_sizes_fxd_pts.extend(matched_values)

   
    # Computing the gold standard for the bit_based model  
    p_n_BMR_intersection = fxd_att_intersection_info(p_fxd, all_att['BMR']['fixed_points'])
    gold_std_basin_sizes_fxd_pts_for_BMR = ([gold_std_basin_sizes_fxd_pts[i] for i in p_n_BMR_intersection[1]]
                                            + [gold_std_basin_sizes_fxd_pts[i] for i in range(len(gold_std_basin_sizes_fxd_pts)) if i not in p_n_BMR_intersection[1]]
                                            + [0]*(len(all_att['BMR']['fixed_points']) - len(p_n_BMR_intersection[1]))
    )
   
    for val in p_n_BMR_intersection[2]:
        matched_values = relev_BMR.loc[relev_BMR['attractor_number'] == val+1, 'attractor_basin_size'].tolist()
        BMR_model_basin_sizes_fxd_pts.extend(matched_values)
    
        
    BMR_model_basin_sizes_fxd_pts = BMR_model_basin_sizes_fxd_pts + [0]*(len(gold_std_basin_sizes_fxd_pts) - len(p_n_BMR_intersection[2]))
    
    BMR_indices_of_the_extra_attr = [ele for ele in np.arange(len(all_att['BMR']['fixed_points'])) if ele not in p_n_BMR_intersection[2]]
    
    for val in BMR_indices_of_the_extra_attr:
        matched_values = relev_BMR.loc[relev_BMR['attractor_number'] == val+1, 'attractor_basin_size'].tolist()
        BMR_model_basin_sizes_fxd_pts.extend(matched_values)
    

    return gold_std_basin_sizes_fxd_pts_for_bio, bio_model_basin_sizes_fxd_pts, gold_std_basin_sizes_fxd_pts_for_IMR, IMR_model_basin_sizes_fxd_pts, gold_std_basin_sizes_fxd_pts_for_BMR, BMR_model_basin_sizes_fxd_pts 

In [10]:
model = 7
fxd_pt_basin_sizes_for_JS_test(model, att_details_publ, att_details_IMR, att_details_BMR)

([4, 28], [4, 28], [4, 28], [16, 16], [4, 28, 0, 0, 0], [20, 4, 4, 2, 2])

In [11]:
def cycle_basin_sizes_for_JS_test(model, att_details_publ, att_details_IMR, att_details_BMR):
    gold_std_basin_sizes_cycles, spurious_bio_basins_cycles,  IMR_model_basin_sizes_cycles, BMR_model_basin_sizes_cycles = [], [], [], []
    
    relev_bio = att_details_publ[att_details_publ['model_number'] == model]
    relev_IMR = att_details_IMR[att_details_IMR['model_number'] == model]
    relev_BMR = att_details_BMR[att_details_BMR['model_number'] == model]
    
    relev_bio_cycles = relev_bio[relev_bio['attractor_length'] != 1]
    relev_IMR_cycles = relev_IMR[relev_IMR['attractor_length'] != 1]
    relev_BMR_cycles = relev_BMR[relev_BMR['attractor_length'] != 1]
          
    biological_cycle_path = f"../input/biological_attractor_files/cyclic_attractor_model_{model}.tsv"
    if os.path.exists(biological_cycle_path):
        cycle_bio_meaning_attract_df = pd.read_csv(biological_cycle_path, sep='\t')
        p_cycle = extract_cyclic_attractors(cycle_bio_meaning_attract_df)
    else:
        p_cycle = []

    all_att = bio_and_thresh_maj_rule_attractors(model)

    # Computing the gold standard for the published model
    p_m_intersection = cyclic_att_intersection(p_cycle, all_att['published']['cycles'])
    gold_standard_indices_in_bio = p_m_intersection[2]
    spurious_bio_att_indices_in_bio = [ele for ele in np.arange(len(all_att['published']['cycles'])) if ele not in gold_standard_indices_in_bio]

    for val in gold_standard_indices_in_bio:
        matched_values = relev_bio_cycles.loc[relev_bio_cycles['attractor_number'] == val+relev_bio_cycles['attractor_number'].iloc[0], 'attractor_basin_size'].tolist()
        gold_std_basin_sizes_cycles.extend(matched_values)

    for val in spurious_bio_att_indices_in_bio:
        matched_values = relev_bio_cycles.loc[relev_bio_cycles['attractor_number'] == val+relev_bio_cycles['attractor_number'].iloc[0], 'attractor_basin_size'].tolist()
        spurious_bio_basins_cycles.extend(matched_values)

    gold_std_basin_sizes_cycles_for_bio = gold_std_basin_sizes_cycles + [0]*len(spurious_bio_basins_cycles)
    bio_model_basin_sizes_cycles = gold_std_basin_sizes_cycles + spurious_bio_basins_cycles


    # Computing the gold standard for the ising model
    p_n_IMR_intersection = cyclic_att_intersection(p_cycle, all_att['IMR']['cycles'])
    gold_std_basin_sizes_cycles_for_IMR = ([gold_std_basin_sizes_cycles[i] for i in p_n_IMR_intersection[1]]
                                        + [gold_std_basin_sizes_cycles[i] for i in range(len(gold_std_basin_sizes_cycles)) if i not in p_n_IMR_intersection[1]]
                                        + [0]*(len(all_att['IMR']['cycles']) - len(p_n_IMR_intersection[1]))
    )

    for val in p_n_IMR_intersection[2]:
        matched_values = relev_IMR_cycles.loc[relev_IMR_cycles['attractor_number'] == val+relev_IMR_cycles['attractor_number'].iloc[0], 'attractor_basin_size'].tolist()
        IMR_model_basin_sizes_cycles.extend(matched_values)

    IMR_model_basin_sizes_cycles = IMR_model_basin_sizes_cycles + [0]*(len(gold_std_basin_sizes_cycles) - len(p_n_IMR_intersection[2]))

    IMR_indices_of_the_extra_attr = [ele for ele in np.arange(len(all_att['IMR']['cycles'])) if ele not in p_n_IMR_intersection[2]]


    for val in IMR_indices_of_the_extra_attr:
        matched_values = relev_IMR_cycles.loc[relev_IMR_cycles['attractor_number'] == val+relev_IMR_cycles['attractor_number'].iloc[0], 'attractor_basin_size'].tolist()
        IMR_model_basin_sizes_cycles.extend(matched_values)


    # Computing the gold standard for the bit_based model
    p_n_BMR_intersection = cyclic_att_intersection(p_cycle, all_att['BMR']['cycles'])
    gold_std_basin_sizes_cycles_for_BMR = ([gold_std_basin_sizes_cycles[i] for i in p_n_BMR_intersection[1]]
                                            + [gold_std_basin_sizes_cycles[i] for i in range(len(gold_std_basin_sizes_cycles)) if i not in p_n_BMR_intersection[1]]
                                            + [0]*(len(all_att['BMR']['cycles']) - len(p_n_BMR_intersection[1]))
    )

    for val in p_n_BMR_intersection[2]:
        matched_values = relev_BMR_cycles.loc[relev_BMR_cycles['attractor_number'] == val+relev_BMR_cycles['attractor_number'].iloc[0], 'attractor_basin_size'].tolist()
        BMR_model_basin_sizes_cycles.extend(matched_values)


    BMR_model_basin_sizes_cycles = BMR_model_basin_sizes_cycles + [0]*(len(gold_std_basin_sizes_cycles) - len(p_n_BMR_intersection[2]))
    BMR_indices_of_the_extra_attr = [ele for ele in np.arange(len(all_att['BMR']['cycles'])) if ele not in p_n_BMR_intersection[2]]

    for val in BMR_indices_of_the_extra_attr:
        matched_values = relev_BMR_cycles.loc[relev_BMR_cycles['attractor_number'] == val+relev_BMR_cycles['attractor_number'].iloc[0], 'attractor_basin_size'].tolist()
        BMR_model_basin_sizes_cycles.extend(matched_values)


    return gold_std_basin_sizes_cycles_for_bio, bio_model_basin_sizes_cycles, gold_std_basin_sizes_cycles_for_IMR, IMR_model_basin_sizes_cycles, gold_std_basin_sizes_cycles_for_BMR, BMR_model_basin_sizes_cycles 

In [12]:
basin_reco_score_dict = {'model_no':[], 'tot_nodes':[], 'max_indeg':[], 'mean_indeg':[], 'pub_gold_std_JS_dist': [], 'IMR_gold_std_JS_dist': [], 'BMR_gold_std_JS_dist':[]}

model_nums = [7,8,10,22,23,31,40,61,63,67,69,74,85,86,88,95,99,100,109,110,133,200,208,212]
for model in model_nums:
    basin_reco_score_dict['model_no'].append(model)

    func_types = pd.read_csv(f'../../network_generation/output/published_network_info/model_{model}/func_type_details_model_{model}.tsv', sep = '\t')
    basin_reco_score_dict['tot_nodes'].append(len(func_types))
    basin_reco_score_dict['max_indeg'].append(int(func_types['No. of inputs'].max()))
    basin_reco_score_dict['mean_indeg'].append(round(func_types['No. of inputs'].mean(), 3))


    fxd_pt_basins = fxd_pt_basin_sizes_for_JS_test(model, att_details_publ, att_details_IMR, att_details_BMR)
    cycle_basins = cycle_basin_sizes_for_JS_test(model, att_details_publ, att_details_IMR, att_details_BMR)

    norm_gold_std_for_bio = np.array(fxd_pt_basins[0] + cycle_basins[0]) / sum(fxd_pt_basins[0] + cycle_basins[0])
    norm_bio_basin = np.array(fxd_pt_basins[1] + cycle_basins[1]) / sum(fxd_pt_basins[1] + cycle_basins[1])

    norm_gold_std_for_IMR = np.array(fxd_pt_basins[2] + cycle_basins[2]) / sum(fxd_pt_basins[2] + cycle_basins[2])
    norm_IMR_basin = np.array(fxd_pt_basins[3] + cycle_basins[3]) / sum(fxd_pt_basins[3] + cycle_basins[3])

    norm_gold_std_for_BMR = np.array(fxd_pt_basins[4] + cycle_basins[4]) / sum(fxd_pt_basins[4] + cycle_basins[4])
    norm_BMR_basin = np.array(fxd_pt_basins[5] + cycle_basins[5]) / sum(fxd_pt_basins[5] + cycle_basins[5])

    basin_reco_score_dict['pub_gold_std_JS_dist'].append(jensenshannon(norm_gold_std_for_bio, norm_bio_basin, base=2))
    basin_reco_score_dict['IMR_gold_std_JS_dist'].append(jensenshannon(norm_gold_std_for_IMR, norm_IMR_basin, base=2))
    basin_reco_score_dict['BMR_gold_std_JS_dist'].append(jensenshannon(norm_gold_std_for_BMR, norm_BMR_basin, base=2))
  
    
df = pd.DataFrame(basin_reco_score_dict)
df.to_csv('../output/basin_recovery_JS_distance.tsv', sep = '\t', index = False)

In [13]:
df

,model_no,tot_nodes,max_indeg,mean_indeg,pub_gold_std_JS_dist,IMR_gold_std_JS_dist,BMR_gold_std_JS_dist
0,7,5,4,2.800,0.000000,0.352500,0.696031
1,8,28,4,1.714,0.528773,0.998085,0.999997
2,10,15,7,2.600,0.561109,1.000000,0.982927
3,22,22,7,2.000,0.948148,0.970777,0.999966
4,23,10,6,3.500,0.000000,1.000000,0.988410
5,31,9,4,2.111,0.753143,1.000000,1.000000
6,40,23,5,1.652,0.910239,0.981832,0.999913
7,61,26,9,3.115,0.665770,1.000000,1.000000
8,63,13,4,1.923,0.255945,0.602344,0.949007
9,67,13,3,1.615,0.416484,0.784859,0.989785
